# Naive Bayes

## Refactoring data

In [2]:
import os, glob
import re

def write_mess2file(messages_path, cur_file, mess_type = "train"):
    classes = []
    for mess_name in glob.glob(os.path.join(messages_path, '*.txt')):
        is_legit = 2
        legit_match = re.match(messages_path + r'/\d*legit\d*\.txt', mess_name)
        
        if legit_match:
            is_legit = 1
        
        if (mess_type == "train"):
            cur_file.write(str(is_legit) + " ")
        else:
            classes.append(is_legit)


        message = open(mess_name, "r")
        subj = message.readline().split()
        rest = message.read().split()
        
        subj_size = len(subj)
        rest_size = len(rest)
        cur_file.write(str(subj_size + rest_size - 1) + " ")
        
        for j in range(1, subj_size): 
            cur_file.write("-" + subj[j] + " ")
        
        for j in range(0, rest_size - 1):
            cur_file.write(rest[j] + " ")
        cur_file.write(rest[rest_size - 1] + "\n")

    return classes


def generate_test_part(test_part, penalties, smooth_intense):
    file_path = '/home/vutaliy/prod/university-courses/MachineLearning/Bayes/test/test{0}'.format(test_part)
    cur_file = open(file_path, "w")
    cur_file.write("2\n")

    for i in range(len(penalties) - 1):
        cur_file.write(str(penalties[i]) + " ")
    cur_file.write(str(penalties[len(penalties)-1]) + "\n") 

    cur_file.write(str(smooth_intense) + "\n")
    mess_count = 0

    test_mess_path = '/home/vutaliy/Downloads/messages/part{0}'.format(test_part)
    
    r, d, f = next(os.walk(test_mess_path))
    test_mess_count = len(f)
    
    for root,dirs,file_ in os.walk('/home/vutaliy/Downloads/messages'):
        mess_count += len(file_)
    cur_file.write(str(mess_count - 1 - test_mess_count) + "\n")

    for i in range(1, 11):
        messages_path = '/home/vutaliy/Downloads/messages/part{0}'.format(i)

        if i == test_part:
            continue
    
        write_mess2file(messages_path, cur_file)
    
    cur_file.write(str(test_mess_count) + "\n")
    classes = write_mess2file(test_mess_path, cur_file, mess_type = "test")

    cur_file.close()

    return classes
            # print(filename)
    # with open(filename, 'r') as f:
    #     text = f.read()
    #     print (filename)
    #     print (len(text))

## Cpp import
```bash
cd src
g++ -c -fPIC "naiveBayes.cpp" -o "naiveBayes.o"
g++ -shared -Wl,-soname,"naiveBayes.so" -o "naiveBayes.so" "naiveBayes.o" 
```

In [1]:
import src.BayesCalling as bc



def gen_all_test_parts():
    for i in range(1,11):
        path_from = "/home/vutaliy/prod/university-courses/MachineLearning/Bayes/test/test1"
        path_to = "/home/vutaliy/prod/university-courses/MachineLearning/Bayes/ans"
        classes = generate_test_part(i, [0.1, 0.2], 0.001)
        bc.naive_bayes(1, path_from, path_to) 

# bc.check(1, path_to)


In [19]:
file_path = '/home/vutaliy/prod/university-courses/MachineLearning/Bayes/test/test{0}'.format(1)
cur_file = open(file_path, "w")

cur_file.writelines([str(1) + " ",str(2) + " ",str(3)])
cur_file.write("12")
cur_file.write("12")

cur_file.close()

In [29]:
# path, dirs, files = 
# print(len(files))
# # print(len(dirs))
size = 0
for root,dirs,file_ in os.walk('/home/vutaliy/Downloads/messages'):
    size += len(file_)
size

1091